<a href="https://colab.research.google.com/github/everythingapplejj/Research-Graph-Embeddings-/blob/JJ/Graph_Isomorphism_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Graph Isomorphism code by Jiwoong Jung (JJ)

In [2]:
!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-{torch.__version__}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-{torch.__version__}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.0/108.0 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.0/210.0 kB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [3]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
#testing visualization
#Implementation by Jiwoong Jung
import torch

#placeholder values:

# Print information about the dataset


# Visualization
import networkx as nx
import matplotlib.pyplot as plt
plt.rcParams['figure.dpi'] = 300
plt.rcParams.update({'font.size': 24})
from torch_geometric.utils import to_networkx
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
"""
Created on Wed Aug 23 12:22:38 2023

@author: vishalr
"""

# we can use the node labels for the first set of the x values
# figure out ways to include diff files for the y_features (1st priority)
# for now, do the y label first,
# later consider the edge features and node features

import pandas as pd
import pickle
import numpy as np


#change this to the folder where you store your data
data_dir = ""

#each of the two data frames below have 20,000 rows, each corresponding to one sample from the original graph
#each sample consists of 21 nodes; node_labels contains the names of these 21 nodes of the form Vxyz
#each node is a DNA fragment of length 500 bases; so Vxyz coveres region [500*xyz, 500*xyz+500)
#adjacency_matrix has the flattended adjacecny matrix for each of these 20,000 samples
#so each row is of dimension 21*21 = 441


#node_labels = np.load(data_dir+'df_chr2R_drosophila_ChIA_Drop_0.1_PASS_20000_MCMC_pivot_sample_node_matrix.npy')

#adjacency_matrix = np.load(data_dir+'df_chr2R_drosophila_ChIA_Drop_0.1_PASS_20000_MCMC_pivot.npy')

worklist_node_labels = []
worklist_adjacency_matrix = []

#importing the rest of the files
# 2L = 0; 2R = 1; 3L = 2, 3R = 3

node_labels_2L = np.load(data_dir+'df_chr2L_drosophila_ChIA_Drop_0.1_PASS_20000_MCMC_pivot_sample_node_matrix.npy')
adjacency_matrix_2L = np.load(data_dir+'df_chr2L_drosophila_ChIA_Drop_0.1_PASS_20000_MCMC_pivot.npy')

worklist_node_labels.append(node_labels_2L)
worklist_adjacency_matrix.append(adjacency_matrix_2L)

node_labels_2R = np.load(data_dir+'df_chr2R_drosophila_ChIA_Drop_0.1_PASS_20000_MCMC_pivot_sample_node_matrix.npy')
adjacency_matrix_2R = np.load(data_dir+'df_chr2R_drosophila_ChIA_Drop_0.1_PASS_20000_MCMC_pivot.npy')

worklist_node_labels.append(node_labels_2R)
worklist_adjacency_matrix.append(adjacency_matrix_2R)

node_labels_3L = np.load(data_dir+'df_chr3L_drosophila_ChIA_Drop_0.1_PASS_20000_MCMC_pivot_sample_node_matrix.npy')
adjacency_matrix_3L = np.load(data_dir+'df_chr3L_drosophila_ChIA_Drop_0.1_PASS_20000_MCMC_pivot.npy')

worklist_node_labels.append(node_labels_3L)
worklist_adjacency_matrix.append(adjacency_matrix_3L)

node_labels_3R = np.load(data_dir+'df_chr3R_drosophila_ChIA_Drop_0.1_PASS_20000_MCMC_pivot_sample_node_matrix.npy')
adjacency_matrix_3R = np.load(data_dir+'df_chr3R_drosophila_ChIA_Drop_0.1_PASS_20000_MCMC_pivot.npy')


worklist_node_labels.append(node_labels_3R)
worklist_adjacency_matrix.append(adjacency_matrix_3R)

In [4]:
for i in range(0,len(worklist_adjacency_matrix)):
  worklist_adjacency_matrix[i] = worklist_adjacency_matrix[i][:,:-1]

In [5]:
print(worklist_node_labels)
print(len(worklist_node_labels))
print(worklist_adjacency_matrix)

[array([[ 5997, 19835,   969, ...,  4455, 27017, 27025],
       [41207, 22040, 41207, ...,   135,   186,   153],
       [13615, 16412, 16377, ..., 41754, 41780, 41767],
       ...,
       [39599,  9995, 38842, ..., 33868, 20856, 14348],
       [39954,  9995, 10131, ...,  2117,  2259,  2251],
       [32509, 19793, 19783, ...,  7313, 42107, 41499]]), array([[34307, 23565,  5042, ...,  5114, 12844, 12993],
       [36450, 36179, 36450, ..., 22359, 28058, 30979],
       [36450, 36179, 33947, ...,  9673, 20209, 12537],
       ...,
       [ 7427, 39513, 19626, ..., 25109, 25119, 27062],
       [33656, 20125, 24230, ..., 28043, 29127, 29091],
       [ 8127, 38554,  7700, ..., 24445, 25489,  3057]]), array([[41331,  1284,  1255, ..., 32082, 18737, 18660],
       [ 1284,  1349,  1363, ...,  3939,  5692, 17395],
       [24386,  5147, 32020, ..., 28096,  6446, 41308],
       ...,
       [ 3375, 19321,  1419, ..., 17371, 28421, 17371],
       [44210,  1306,  3592, ..., 11490,  1774, 35930],
       

In [6]:
# proceeding with data cleaning:
import numpy as np
def remove_duplicates(node_labels, adj_matrix):
  sample_nodes = node_labels
  sample_adj_matrix = adj_matrix
  i = 0
  j = i + 1
  while(i < len(sample_nodes) - 1):
    if(sample_nodes[i] == sample_nodes[j]):
      sample_nodes = np.delete(sample_nodes,j)
      sample_adj_matrix = np.delete(np.delete(sample_adj_matrix, j, 0), j, 1)
      if(j >= len(sample_nodes)):
        i = i + 1
        j = i + 1
      continue
    else:
      j = j + 1
      if(j >= len(sample_nodes)):
        i = i + 1
        j = i + 1
  return sample_nodes, sample_adj_matrix

In [7]:
# algorithm for subgraph delete duplicate
worklist_new_adj_matrix = []
worklist_new_node_labels = []

counter = 0
label_number = 0

for adjacency_matrix in worklist_adjacency_matrix:
  sample_graphs = ""
  sample_nodes = ""
  adj_matrix = []
  node_labels = []
  for graph in adjacency_matrix:
    sample_graph = graph.reshape(21,21)
    sample_node = worklist_node_labels[label_number][counter]
    sample_node, sample_graphs = remove_duplicates(sample_node, sample_graph)
    adj_matrix.append(sample_graphs)
    node_labels.append(sample_node)
    counter = counter + 1
  worklist_new_adj_matrix.append(adj_matrix)
  worklist_new_node_labels.append(node_labels)
  counter = 0
  label_number = label_number + 1

The following code is the logic and algorithm for checking if two graphs are isomorphic or not. We are using networkx library, and it is using state space search algorithm, which takes the bijection of each node along with backtrackking method to check for isomorphism.

In [8]:
import networkx as nx
def is_isomorphic(node_labels1, adj_matrix1, node_labels2, adj_matrix2):
    nx_converter_1 = np.array(adj_matrix1) # we need to convert the adj matrix to nx readable
    nx_converter_2 = np.array(adj_matrix2)
    G1 = nx.from_numpy_array(nx_converter_1)
    G2 = nx.from_numpy_array(nx_converter_2)
    nx.set_node_attributes(G1, dict(zip(G1, node_labels1)), "label") # we need to zip each of the node_labels to each specific nodes
    nx.set_node_attributes(G2, dict(zip(G2, node_labels2)), "label")
    return nx.vf2pp_is_isomorphic(G1, G2, node_label = "label") # utilize the library function

In [9]:
# lets check with some of the test cases!
test_cases = ([
    {
        "node_labels1": ['A', 'B', 'C'],
        "node_labels2": ['A', 'B', 'C'],
        "adj_matrix1": [
            [0, 1, 1],
            [1, 0, 0],
            [1, 0, 0]
        ],
        "adj_matrix2": [
            [0, 1, 1],
            [1, 0, 0],
            [1, 0, 0]
        ],
        "is_isomorphic": True
    },
    {
        "node_labels1": ['A', 'B', 'C'],
        "node_labels2": ['X', 'Y', 'Z'],
        "adj_matrix1": [
            [0, 1, 1],
            [1, 0, 1],
            [1, 1, 0]
        ],
        "adj_matrix2": [
            [0, 1, 0],
            [1, 0, 1],
            [0, 1, 0]
        ],
        "is_isomorphic": False
    },
    {
        "node_labels1": [1],
        "node_labels2": [1],
        "adj_matrix1": [
            [0]
        ],
        "adj_matrix2": [
            [0]
        ],
        "is_isomorphic": True
    },
    {
        "node_labels1": [1, 2, 3, 4],
        "node_labels2": [1, 2, 3, 4],
        "adj_matrix1": [
            [0, 1, 1, 0],
            [1, 0, 0, 1],
            [1, 0, 0, 1],
            [0, 1, 1, 0]
        ],
        "adj_matrix2": [
            [0, 1, 0, 1],
            [1, 0, 1, 0],
            [0, 1, 0, 1],
            [1, 0, 1, 0]
        ],
        "is_isomorphic": False
    },
    {
        "node_labels1": [1, 2, 3],
        "node_labels2": [1, 2, 3, 4],
        "adj_matrix1": [
            [0, 1, 1],
            [1, 0, 0],
            [1, 0, 0]
        ],
        "adj_matrix2": [
            [0, 1, 1, 0],
            [1, 0, 0, 1],
            [1, 0, 0, 1],
            [0, 1, 1, 0]
        ],
        "is_isomorphic": False
    }
])

# this is a tester. I am checking if my algorithm actually works.
def test_isomorphism_checker(is_isomorphic, test_cases):
    for i, test_case in enumerate(test_cases):
        result = is_isomorphic(test_case["node_labels1"], test_case["adj_matrix1"], test_case["node_labels2"], test_case["adj_matrix2"])
        assert result == test_case["is_isomorphic"], f"Test case {i+1} failed: expected {test_case['is_isomorphic']}, got {result}"
    print("All test cases passed!")

test_isomorphism_checker(is_isomorphic, test_cases)

All test cases passed!


Good! All test cases has passed. We can add in more test cases later on (maybe think aboyt some edge case) but I believe about the accuracy on the built in function. Lets visualize our datas now.

In [ ]:
iso_node_labels = []
iso_adj_matrix = []

adj_collector = []
node_labels_collector = []

counter_out = 0
for chromosome in worklist_new_node_labels:
  counter_in = 0
  for graphs in chromosome:
    node_labels_collector.append(graphs)
    adj_collector.append(worklist_new_adj_matrix[counter_out][counter_in])
    counter_in += 1
  counter_out += 1

# is_isomorphic(node_labels1, adj_matrix1, node_labels2, adj_matrix2)

#pace_inside = 0
pace_outside = 2
tester = 0
for i in range(pace_outside, 20000 + pace_outside):
  for j in range(20000 + pace_outside, len(node_labels_collector)):
    if(len(node_labels_collector[i]) != len(node_labels_collector[j])):
      continue
    #print("Passing First Filter")
    if(is_isomorphic(node_labels_collector[i], adj_collector[i], node_labels_collector[j], adj_collector[j])):
      iso_node_labels.append(np.array([node_labels_collector[i],node_labels_collector[j]]))
      iso_adj_matrix.append(np.array([adj_collector[i], adj_collector[j]]))
      tester = 1
    if(tester == 1):
      break
  if(tester == 1):
    break
  if (i + 1 == 20000 + pace_outside):
    print("Passing Filters")
    pace_outside += 20000
    if(pace_outside == len(node_labels_collector)):
      break

print(len(iso_node_labels))
print(len(iso_adj_matrix))

In [12]:
print(len(node_labels_collector))

80000
